In [2]:
!pip install scikit_posthocs -q


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [1]:
import sqlite3
import ast
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ana_utils import convert_listlike_cols, db_path, font_size, font_size_large, df_to_latex_table
import json

In [2]:
METRIC = "r2_score"

In [3]:
def sec2str(seconds):
    if seconds < 60:
        return str(seconds) + "s"
    elif seconds < 60*60:
        return str(seconds // 60) + "min"
    else:
        return str(seconds // (60*60)) + "h"

conn = sqlite3.connect(db_path)
#df = pd.read_sql(f"SELECT model, search_space_hash, {METRIC}, data_config_hash, data_id, search_space_name FROM results ", conn)
df = pd.read_sql(f"SELECT results.model, {METRIC}, data_config_hash, data_id, json_extract(search_spaces.config, '$.fit_kwargs') as fit_kwargs FROM results join search_spaces on results.search_space_hash == search_spaces.hash_key", conn)
conn.close()
df = convert_listlike_cols(df)
#list(df[df["model"]=="AutoGluon"]["config"])[0]
df["Model"] = df[["model", "fit_kwargs"]].apply(lambda x: x["model"] if x["fit_kwargs"] is None else f"{x['model']} ({sec2str(json.loads(x['fit_kwargs'])['time_limit'])})", axis=1)
df = df.drop(["fit_kwargs"], axis=1)
df[df["model"] == "AutoGluon"]

r2_score


,model,r2_score,data_config_hash,data_id,Model
625,AutoGluon,"[0.5574375389941029, 0.5826927947924572, 0.616...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,0,AutoGluon (1min)
627,AutoGluon,"[0.9960804703979276, 0.9953001729685682, 0.998...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,2,AutoGluon (1min)
628,AutoGluon,"[0.965963201240902, 0.9733752271736906, 0.9520...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,1,AutoGluon (1min)
629,AutoGluon,"[0.908430087226909, 0.9217241194669571, 0.9308...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,5,AutoGluon (1min)
630,AutoGluon,"[0.3881788329195903, 0.33683931727050687, 0.42...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,6,AutoGluon (1min)
...,...,...,...,...,...
1049,AutoGluon,"[0.9096488229980708, 0.9605343163723057, 0.959...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,3,AutoGluon (2h)
1051,AutoGluon,"[0.7069392025103225, 0.6555009134261928, 0.662...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,19,AutoGluon (2h)
1054,AutoGluon,"[0.6225505947683183, 0.6368267850156144, 0.607...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,9,AutoGluon (2h)
1058,AutoGluon,"[0.808253160013988, 0.7638861419210315, 0.7623...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,34,AutoGluon (2h)


In [4]:
df['Model'].value_counts()

Model
Dummy                      35
AutoGluon (2h)             35
CatBoost                   35
ElasticNet                 35
DecisionTree               35
LightGBM                   35
PassiveAggressive          35
M5                         35
SGD                        35
AutoGluon (5min)           35
AutoGluon (1min)           35
AutoGluon (10min)          35
MLP                        35
AutoGluon (30min)          34
Huber                      34
Bagging                    34
BayesianRidge              34
KNN                        34
PartialLeastSquares        34
ExtraTrees                 34
GradientBoosting           34
Lasso                      34
LARS                       34
RidgeRegressor             34
AutoGluon (1h)             33
AdaBoost                   32
GaussianProcess            32
LassoLars                  31
RandomForest               29
Tweedie                    29
XGBoost                    28
PolynomialRegression       25
SupportVectorRegression     9
Name

In [5]:
df["avg_metric"] = df[METRIC].apply(
    lambda x: sum(x) / len(x) if x else None
)
df

,model,r2_score,data_config_hash,data_id,Model,avg_metric
0,AdaBoost,"[0.9339437124785132, 0.9190862929848298, 0.901...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,1,AdaBoost,0.917481
1,AdaBoost,"[0.9955497053557076, 0.9912188846935301, 0.993...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,2,AdaBoost,0.993115
2,AdaBoost,"[0.875016211574677, 0.9268958030218877, 0.9151...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,3,AdaBoost,0.905170
3,AdaBoost,"[0.5027258214779751, 0.5364031371526645, 0.532...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,0,AdaBoost,0.514758
4,AdaBoost,"[0.10229725235510645, 0.05565964452376304, -0....",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,7,AdaBoost,-0.016382
...,...,...,...,...,...,...
1072,MLP,"[0.9875149989881462, 0.9886374583399155, 0.983...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,14,MLP,0.985943
1073,RandomForest,"[0.9822109304756447, 0.9917075915682768, 0.989...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,12,RandomForest,0.986228
1074,MLP,"[0.5091186131327867, 0.9923297805717298, 0.999...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,13,MLP,0.915528
1075,RandomForest,"[0.9803651556410763, 0.9797405046930209, 0.978...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,14,RandomForest,0.981098


In [6]:
# Name 
# Avg Normalized Performance
# Avg Normalized Performance (Without AG) 
# Avg Rank
# Avg Rank (Without AG) ~ To answer is the weakest AG already the best method
# Avg Absolute Inference Time per sample

In [8]:
groups = df.groupby(by=["data_config_hash", "data_id"])
group_collection = []
all_models = df["Model"].unique()

for (hash, data_id), group_df in groups:
    # Because if worse than zero we dont really care by how much
    group_df.loc[group_df["avg_metric"] < 0, "avg_metric"] = 0
    # Set Missing Entries to 0 - they likely crahsed
    group_df = group_df.set_index(group_df["Model"])
    group_df = group_df[~group_df.index.duplicated(keep="first")]
    group_df = group_df.drop(["Model"], axis=1)
    group_df = group_df.reindex(all_models, fill_value=0)
    group_df["Model"] = group_df.index
    group_df = group_df.reset_index(drop=True)
    
    maximum = group_df["avg_metric"].max()
    median = group_df["avg_metric"].median()
    if maximum <= 0.001:
        group_df["normalized_metric"] = 0 # If all fail, all fail simple as that
    else:
        group_df["normalized_metric"] = (group_df["avg_metric"] - median) / (maximum-median)
    assert group_df["normalized_metric"].max() < 1.2
    group_df["rank"] = group_df["avg_metric"].rank(method="dense", ascending=False)
    group_df["rounded_rank"] = group_df["avg_metric"].round(2).rank(method="dense", ascending=False)
    group_df["losses(tied)"] = (group_df["rounded_rank"] == group_df["rounded_rank"].max())
    group_df["true losses(tied)"] = (group_df["rounded_rank"] == group_df["rounded_rank"].max()) & (group_df["rounded_rank"] != group_df["rounded_rank"].min())

    group_df_wo_ag = group_df[group_df["model"] != "AutoGluon"]
    maximum_wo_ag = group_df_wo_ag["avg_metric"].max()
    median_wo_ag = group_df_wo_ag["avg_metric"].median()
    if maximum_wo_ag <= 0.001:
        group_df["normalized_metric_wo_ag"] = 0 # If all fail, all fail simple as that
    else:
        group_df["normalized_metric_wo_ag"] = (group_df["avg_metric"] - median_wo_ag) / (maximum_wo_ag-median_wo_ag)
        
    #group_df["rank_wo_ag"] = group_df["avg_metric"].rank(ascending=False)

    gavg = (
        group_df.groupby("model", as_index=False)
          .agg(avg_metric=("avg_metric", "max"))
    )
    
    # 2) rank groups by that average
    gavg["rank_wo_ag"] = gavg["avg_metric"].rank(method="dense", ascending=False)
    gavg["rank_wo_ag_round"] = gavg["avg_metric"].round(2).rank(method="dense", ascending=False)
    rank_wo_ag_round = gavg[gavg["model"]!="AutoGluon"]["avg_metric"].round(2).rank(method="dense", ascending=False)
    gavg["victories_wo_ag(ties)"] = (rank_wo_ag_round == 1)
    gavg["victories_wo_ag"] = gavg[gavg["model"]!="AutoGluon"]["avg_metric"].rank(method="dense", ascending=False) == 1
    
    
    # 3) merge back — no restructuring of df
    group_df = group_df.merge(gavg[["model", "rank_wo_ag", "rank_wo_ag_round", "victories_wo_ag", "victories_wo_ag(ties)"]], on="model", how="left")
    #print(group_df[group_df["model"] == "AutoGluon"]["rank_wo_ag"])
    if list(group_df[group_df["model"] == "AutoGluon"]["rank_wo_ag_round"])[0] > 1:
        print(maximum, gavg[gavg["model"] == "AutoGluon"][["rank_wo_ag_round", "avg_metric"]])
    
    autogluon_names = group_df[group_df["model"]=="AutoGluon"]["Model"]
    ag_max_perf = 0
    for ag_name in autogluon_names:
        ag_perf = group_df[group_df["Model"] == ag_name]["avg_metric"].iloc[0]
        group_df[f"victories_against_{ag_name}"] = group_df["avg_metric"] > ag_perf
        group_df[f"major_victories_against_{ag_name}"] = group_df["avg_metric"] > (ag_perf*1.01)
        if ag_perf > ag_max_perf:
            ag_max_perf = ag_perf
    group_df[f"victories_against_best_ag"] = group_df["avg_metric"] > ag_max_perf
    group_df[f"major_victories_against_best_ag"] = group_df["avg_metric"] > (ag_max_perf*1.01)
    

    group_df["failures"] = group_df["avg_metric"] <= 0.001    
    group_df = group_df.reset_index(drop=False)
    # TODO Victories against AG (x/y/z/W)
    # TODO Victories without AG
    # best_models = list(group_df[group_df["avg_metric"] >= maximum*0.999]["model"])
    # if not "AutoGluon" in best_models:
        
    #     print(maximum, best_models)
    #     print(group_df[group_df["model"] == "AutoGluon"]["avg_metric"])
    group_collection.append(group_df)
df_ = pd.concat(group_collection)
df_ = df_.groupby(by=["Model"]).agg({
    "avg_metric": "mean",
    "rank": "mean",
    "rounded_rank": "mean",
    "rank_wo_ag": "mean", # Without counting AG several times
    "rank_wo_ag_round": "mean",
    "normalized_metric": "mean",
    "normalized_metric_wo_ag": "mean",
    "victories_wo_ag": "sum",
    "victories_against_AutoGluon (5min)": "sum",
    "victories_against_AutoGluon (30min)": "sum",
    "victories_against_AutoGluon (1h)": "sum",
    "major_victories_against_AutoGluon (1h)": "sum",
    "victories_against_AutoGluon (2h)": "sum",
    "victories_against_best_ag": "sum",
    "major_victories_against_best_ag": "sum",
    "losses(tied)": "sum",
    "true losses(tied)": "sum",
    "failures": "sum",
})

df_.sort_values(by=["normalized_metric_wo_ag", "rank"])

0.536635986414808    rank_wo_ag_round  avg_metric
2               2.0    0.525376
0.9244292560369967    rank_wo_ag_round  avg_metric
2               2.0    0.899812
0.7405851337893253    rank_wo_ag_round  avg_metric
2               4.0    0.665532


,avg_metric,rank,rounded_rank,rank_wo_ag,rank_wo_ag_round,normalized_metric,normalized_metric_wo_ag,victories_wo_ag,victories_against_AutoGluon (5min),victories_against_AutoGluon (30min),victories_against_AutoGluon (1h),major_victories_against_AutoGluon (1h),victories_against_AutoGluon (2h),victories_against_best_ag,major_victories_against_best_ag,losses(tied),true losses(tied),failures
Model,,,,,,,,,,,,,,,,,,
Dummy,0.000000,28.257143,13.057143,23.485714,12.314286,-30.915550,-25.565309,1,0,0,0,0,0,0,0,35,34,35
SupportVectorRegression,0.072748,25.685714,11.600000,20.942857,10.857143,-30.230237,-24.809979,1,0,0,0,0,0,0,0,29,28,29
PolynomialRegression,0.387046,20.000000,8.885714,15.400000,8.142857,-25.219125,-22.579730,4,1,1,1,0,1,1,0,13,12,13
Tweedie,0.374601,21.914286,9.285714,17.142857,8.542857,-24.157670,-21.860515,1,0,0,0,0,0,0,0,9,8,9
RandomForest,0.617464,12.942857,5.000000,8.657143,4.457143,-22.840247,-21.294688,3,2,3,3,2,3,1,1,6,5,6
ExtraTrees,0.690384,11.000000,4.000000,6.857143,3.514286,-20.356631,-20.180971,7,3,3,3,2,3,2,2,2,1,2
DecisionTree,0.604838,19.342857,7.914286,14.600000,7.200000,-8.511119,-8.190588,1,0,0,0,0,0,0,0,4,3,4
Bagging,0.557385,20.000000,8.228571,15.228571,7.485714,-5.073801,-4.597331,1,0,0,0,0,0,0,0,1,0,1
KNN,0.527012,19.085714,7.942857,14.342857,7.228571,-4.616884,-4.119409,1,0,0,0,0,0,0,0,2,1,2


In [10]:
df_["Model"] = df_.index

df_to_latex_table(
    df=df_[["Model", "normalized_metric_wo_ag", "normalized_metric", "rounded_rank", "true losses(tied)", "failures"]],
    filepath="./cache/ranking.tex",
    caption="Performance comparison of all evaluated methods against AutoGluon",
    label="tab:ranking",
    float_fmt=".2f",
)

'\\begin{table}[t]\n\\centering\n\\begin{tabular}{lrrrrr}\n\\toprule\n Model                   &   normalized\\_metric\\_wo\\_ag &   normalized\\_metric &   rounded\\_rank &   true losses(tied) &   failures \\\\\n\\midrule\n AdaBoost                &                      0.36 &                0.17 &           5.4  &                   3 &          4 \\\\\n AutoGluon (10min)       &                      0.92 &                0.82 &           1.66 &                   0 &          1 \\\\\n AutoGluon (1h)          &                      1    &                0.91 &           1.51 &                   0 &          1 \\\\\n AutoGluon (1min)        &                      0.21 &                0.07 &           2.26 &                   0 &          1 \\\\\n AutoGluon (30min)       &                      1    &                0.91 &           1.49 &                   0 &          1 \\\\\n AutoGluon (4h)          &                      0.95 &                0.85 &           1.66 &                  

In [8]:
# TODO 
# Get Names of outlier DS describe what happened (fair but minor, fair server but only two models better, all shit)

In [13]:
import scikit_posthocs as sp
import matplotlib.pyplot as plt

fig = sp.critical_difference_diagram(
    df.values,
    df.columns.tolist(),
    alpha=0.05,  # Nemenyi
)
plt.show()


TypeError: critical_difference_diagram() got an unexpected keyword argument 'alpha'